In [ ]:
import numpy as np

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# world.reset()

In [ ]:
import getpass
user = getpass.getuser()

In [ ]:
from omni.isaac.kit import SimulationApp

usd_path = f"omniverse://localhost/Users/{user}/test_hand2.usd"


simulation_app = SimulationApp({"headless": True, "open_usd": usd_path, "livesync_usd": usd_path})


In [ ]:
from omni.isaac.core import World
world = World()

In [ ]:
from hand_env import HandEnv

# env = HandEnv("/World/Hand/Bones/l_carpal_mid", "/World/Hand*/Bones/l_thumbSkeleton_grp/l_distalThumb_mid")
env = HandEnv("/World/Hand/Bones", "/World/Hand*/Bones/l_thumbSkeleton_grp/l_distalThumb_mid")

In [ ]:
world.reset()

In [ ]:
env.start()
world.scene.add(env.robots)

In [ ]:
hand_pos, hand_rot = env.robots.get_world_poses()

In [ ]:
hand_pos, hand_rot

In [ ]:
# hand_pos[...,0] += 10
# hand_rot = np.array([[1,0,0,0]])

In [ ]:
from utils import get_bounding_box

In [ ]:
keyword = "handle_17"
stage = world.scene.stage
prim_list = list(stage.TraverseAll())
prim_list = [ item for item in prim_list if keyword in item.GetPath().pathString and item.GetTypeName() == 'Mesh' ]

bboxes_list  = []
for prim in prim_list:
    bboxes = get_bounding_box(prim.GetPath().pathString)
    bboxes_list.append(bboxes)

In [ ]:
bboxes_list[1]

In [ ]:
center = 0.5 * (bboxes_list[1][0] + bboxes_list[1][1])

In [ ]:
hand_pos = np.array([[center[0] - 7, center[1], center[2] - 2]])

In [ ]:
hand_pos, hand_rot

In [ ]:
env.robots.set_world_poses(hand_pos, hand_rot)
world.step(render=True)

In [ ]:
# pos = np.array([[ 0.5, -0., 0.5]], dtype=np.float32)
# env.robots.set_world_poses(positions=pos)

In [ ]:
world.reset()

In [ ]:
world.step(render=True)

In [ ]:
env.robots.get_joint_positions()

In [ ]:
env.robots._physics_view.dof_paths

In [ ]:
pause

In [ ]:
import time

In [ ]:
for i in range(20):
    pos = np.array([
        [ 0,  0, 
          i * 0.01,  0,  
          0, 0, 
         -i * 0.02, 0,  
         -i * 0.01, 0,  
         0,  -i * 0.03,  -i * 0.03,  -i * 0.03,  -i * 0.03,
         0,  -i * 0.03,  -i * 0.03,  -i * 0.03,  -i * 0.03, 
        ],
    ])

    # pos = np.random.randn(2,25)
    time.sleep(0.2)
    env.robots.set_joint_positions(pos) # 
    world.step(render=True)

# Stage one

In [ ]:
i = 30
pos = np.array([
        [ 0,  0, 
          i * 0.01,  0,  
          0, 0, 
         -i * 0.02, 0,  
         -i * 0.01, 0,  
          0, 0, 0, 0, 0, # 0,  -i * 0.03,  -i * 0.03,  -i * 0.03,  -i * 0.03,
          0, 0, 0, 0, 0, # 0,  -i * 0.03,  -i * 0.03,  -i * 0.03,  -i * 0.03, 
        ],
    ])

for _ in range(30):
    env.robots.set_joint_position_targets(pos) 
    world.step(render=True)

# Stage two

In [ ]:
i = 30
pos = np.array([
        [ 0,  0, 
          i * 0.01,  0,  
          0, 0, 
         -i * 0.02, 0,  
         -i * 0.01, 0,  
          0,  -i * 0.03,  -i * 0.03,  -i * 0.03,  -i * 0.03,
          0,  -i * 0.03,  -i * 0.03,  -i * 0.03,  -i * 0.03, 
        ],
    ])

for _ in range(30):
    env.robots.set_joint_position_targets(pos) 
    world.step(render=True)

# Stage three

In [ ]:
for _ in range(100):
    env.robots.set_joint_position_targets(pos) 
    
    hand_pos, hand_rot = env.robots.get_world_poses()
    hand_pos[...,0] += 0.01
    env.robots.set_world_poses(hand_pos, hand_rot)
    
    world.step(render=True)

------------------

In [ ]:
env.robots.get_joint_positions()

In [ ]:
joint_v = np.array([[
     1,  0,  
     0., 0.,
     0., 0., 
     0., 0., 
     0., 0., 
     0, 0, 0, 0, 0,
     0, 0, 0, 0, 0
]]).transpose()

In [ ]:
joint_v.shape

In [ ]:
jacobians = env.robots._physics_view.get_jacobians()

In [ ]:
jacobians.shape

In [ ]:
delta_pos_rot = np.matmul(jacobians[0], joint_v)
for i in range(len(delta_pos_rot)):
    print(i, delta_pos_rot[i].transpose())

# Jacobian

In [ ]:
world.reset()
world.step(render=True)

In [ ]:
thumb_pos, thumb_rot = env.xforms.get_world_poses()

In [ ]:
thumb_rot = thumb_rot[:,[1,2,3,0]] # WXYZ -> XYZW

In [ ]:
thumb_pos, thumb_rot

In [ ]:
target_pos = thumb_pos.copy()#np.array([[-0.37809137,  -0.3650283  , 0.05804292]])

In [ ]:
target_pos[...,1] -= 0.5

In [ ]:
target_pos

In [ ]:
env.move_finger_to_fast(target_pos, thumb_rot, world, finger = "thumb")

In [ ]:
target_pos = np.zeros((env.num_envs, 20)) 
# target_pos = np.random.randn(env.num_envs, 20) 

In [ ]:
env.num_envs

In [ ]:
target_pos

In [ ]:
for _ in range(100):
    world.step(render=True)
    thumb_pos, thumb_rot = env.xforms.get_world_poses()
    env.robots.set_joint_position_targets(target_pos) 
    print("thumb_pos", thumb_pos)

In [ ]:
env.robots.get_joint_positions()

# Control

In [ ]:
pos, rot = env.robots.get_local_poses()

In [ ]:
pos, rot

In [ ]:
# pos[...,0] = -10

In [ ]:
env.robots.set_local_poses(pos,rot)
world.step(render=True)

In [ ]:
hand_prim = world.scene.stage.GetPrimAtPath("/World/Hand")

In [ ]:
from pxr import UsdGeom, Gf

In [ ]:
mat = Gf.Matrix4f().SetTranslate(Gf.Vec3f(0, 0, 0))

In [ ]:
import omni

In [ ]:
omni.kit.commands.execute(
    "TransformPrimCommand",
    path="/World/Hand",
    new_transform_matrix=mat,
)

In [ ]:
world.reset()